In [133]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from tqdm import tqdm_notebook
import lightgbm as lgb
from catboost import Pool, CatBoostClassifier
import xgboost as xgb
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings("ignore")

import seaborn as sns



plt.style.use('seaborn')
sns.set(font_scale=1)

import gc

In [134]:
train = pd.read_csv("../input/landslide-prevention-and-innovation-challenge/Train (12).csv")
test = pd.read_csv("../input/landslide-prevention-and-innovation-challenge/Test (13).csv")
sample = pd.read_csv('../input/landslide-prevention-and-innovation-challenge/SampleSubmission (7).csv')

In [135]:
train["Label"].value_counts()

0    8148
1    2716
Name: Label, dtype: int64

In [136]:
train

,Sample_ID,1_elevation,2_elevation,3_elevation,4_elevation,5_elevation,6_elevation,7_elevation,8_elevation,9_elevation,...,17_sdoif,18_sdoif,19_sdoif,20_sdoif,21_sdoif,22_sdoif,23_sdoif,24_sdoif,25_sdoif,Label
0,1,130,129,127,126,123,126,125,124,122,...,1.281779,1.281743,1.281720,1.281684,1.281811,1.281788,1.281752,1.281729,1.281693,0
1,2,161,158,155,153,151,162,159,155,153,...,1.359639,1.359608,1.359587,1.359556,1.359683,1.359662,1.359631,1.359610,1.359579,1
2,3,149,151,154,156,158,154,157,158,160,...,1.365005,1.365025,1.365055,1.365075,1.364937,1.364967,1.364988,1.365018,1.365038,0
3,4,80,78,77,75,73,80,78,77,75,...,1.100708,1.100738,1.100759,1.100789,1.100630,1.100650,1.100680,1.100700,1.100731,0
4,5,117,115,114,112,110,115,113,111,110,...,1.284180,1.284130,1.284056,1.284006,1.284125,1.284050,1.284001,1.283926,1.283876,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,10860,203,202,201,200,199,202,201,201,199,...,1.271912,1.272011,1.272076,1.272174,1.271720,1.271786,1.271884,1.271950,1.272048,0
10860,10861,432,430,427,423,421,431,429,426,422,...,1.331153,1.331137,1.331113,1.331096,1.331207,1.331183,1.331167,1.331142,1.331126,1
10861,10862,348,345,342,339,335,344,342,339,336,...,1.298362,1.298337,1.298321,1.298297,1.298392,1.298376,1.298352,1.298336,1.298311,1
10862,10863,93,91,90,88,86,96,94,93,92,...,1.355035,1.355061,1.355099,1.355125,1.354937,1.354976,1.355002,1.355041,1.355067,0


In [137]:
id_=test["Sample_ID"]
test.drop("Sample_ID", axis=1, inplace=True)
train.drop("Sample_ID", axis=1, inplace=True)

In [138]:
col=train.columns[:-1]

In [139]:
col

Index(['1_elevation', '2_elevation', '3_elevation', '4_elevation',
       '5_elevation', '6_elevation', '7_elevation', '8_elevation',
       '9_elevation', '10_elevation',
       ...
       '16_sdoif', '17_sdoif', '18_sdoif', '19_sdoif', '20_sdoif', '21_sdoif',
       '22_sdoif', '23_sdoif', '24_sdoif', '25_sdoif'],
      dtype='object', length=225)

In [140]:
train["elevation_mean"]=train[train.columns[:25].to_list()].mean(axis=1)
train["slope_mean"]=train[train.columns[25:50].to_list()].mean(axis=1)
train["aspect_mean"]=train[train.columns[50:75].to_list()].mean(axis=1)
train["placurv_mean"]=train[train.columns[75:100].to_list()].mean(axis=1)
train["procurv_mean"]=train[train.columns[100:125].to_list()].mean(axis=1)
train["lsfactor_mean"]=train[train.columns[125:150].to_list()].mean(axis=1)
train["twi_mean"]=train[train.columns[150:175].to_list()].mean(axis=1)
train["geology_mean"]=train[train.columns[175:200].to_list()].mean(axis=1)
train["sdoif_mean"]=train[train.columns[200:225].to_list()].mean(axis=1)
########################################################################
train["elevation_count"]=train[train.columns[:25].to_list()].count(axis=1)
train["slope_count"]=train[train.columns[25:50].to_list()].count(axis=1)
train["aspect_count"]=train[train.columns[50:75].to_list()].count(axis=1)
train["placurv_count"]=train[train.columns[75:100].to_list()].count(axis=1)
train["procurv_count"]=train[train.columns[100:125].to_list()].count(axis=1)
train["lsfactor_count"]=train[train.columns[125:150].to_list()].count(axis=1)
train["twi_count"]=train[train.columns[150:175].to_list()].count(axis=1)
train["geology_count"]=train[train.columns[175:200].to_list()].count(axis=1)
train["sdoif_count"]=train[train.columns[200:225].to_list()].count(axis=1)
#########################################################################
train["elevation_std"]=train[train.columns[:25].to_list()].std(axis=1)
train["slope_std"]=train[train.columns[25:50].to_list()].std(axis=1)
train["aspect_std"]=train[train.columns[50:75].to_list()].std(axis=1)
train["placurv_std"]=train[train.columns[75:100].to_list()].std(axis=1)
train["procurv_std"]=train[train.columns[100:125].to_list()].std(axis=1)
train["lsfactor_std"]=train[train.columns[125:150].to_list()].std(axis=1)
train["twi_std"]=train[train.columns[150:175].to_list()].std(axis=1)
train["geology_std"]=train[train.columns[175:200].to_list()].std(axis=1)
train["sdoif_std"]=train[train.columns[200:225].to_list()].std(axis=1)
#########################################################################
train["elevation_min"]=train[train.columns[:25].to_list()].min(axis=1)
train["slope_min"]=train[train.columns[25:50].to_list()].min(axis=1)
train["aspect_min"]=train[train.columns[50:75].to_list()].min(axis=1)
train["placurv_min"]=train[train.columns[75:100].to_list()].min(axis=1)
train["procurv_min"]=train[train.columns[100:125].to_list()].min(axis=1)
train["lsfactor_min"]=train[train.columns[125:150].to_list()].min(axis=1)
train["twi_min"]=train[train.columns[150:175].to_list()].min(axis=1)
train["geology_min"]=train[train.columns[175:200].to_list()].min(axis=1)
train["sdoif_min"]=train[train.columns[200:225].to_list()].min(axis=1)
#########################################################################
train["elevation_max"]=train[train.columns[:25].to_list()].max(axis=1)
train["slope_max"]=train[train.columns[25:50].to_list()].max(axis=1)
train["aspect_max"]=train[train.columns[50:75].to_list()].max(axis=1)
train["placurv_max"]=train[train.columns[75:100].to_list()].max(axis=1)
train["procurv_max"]=train[train.columns[100:125].to_list()].max(axis=1)
train["lsfactor_max"]=train[train.columns[125:150].to_list()].max(axis=1)
train["twi_max"]=train[train.columns[150:175].to_list()].max(axis=1)
train["geology_max"]=train[train.columns[175:200].to_list()].max(axis=1)
train["sdoif_max"]=train[train.columns[200:225].to_list()].max(axis=1)
#########################################################################
train["elevation_kurt"]=train[train.columns[:25].to_list()].kurt(axis=1)
train["slope_kurt"]=train[train.columns[25:50].to_list()].kurt(axis=1)
train["aspect_kurt"]=train[train.columns[50:75].to_list()].kurt(axis=1)
train["placurv_kurt"]=train[train.columns[75:100].to_list()].kurt(axis=1)
train["procurv_kurt"]=train[train.columns[100:125].to_list()].kurt(axis=1)
train["lsfactor_kurt"]=train[train.columns[125:150].to_list()].kurt(axis=1)
train["twi_kurt"]=train[train.columns[150:175].to_list()].kurt(axis=1)
train["geology_kurt"]=train[train.columns[175:200].to_list()].kurt(axis=1)
train["sdoif_kurt"]=train[train.columns[200:225].to_list()].kurt(axis=1)
#########################################################################
train["elevation_skew"]=train[train.columns[:25].to_list()].skew(axis=1)
train["slope_skew"]=train[train.columns[25:50].to_list()].skew(axis=1)
train["aspect_skew"]=train[train.columns[50:75].to_list()].skew(axis=1)
train["placurv_skew"]=train[train.columns[75:100].to_list()].skew(axis=1)
train["procurv_skew"]=train[train.columns[100:125].to_list()].skew(axis=1)
train["lsfactor_skew"]=train[train.columns[125:150].to_list()].skew(axis=1)
train["twi_skew"]=train[train.columns[150:175].to_list()].skew(axis=1)
train["geology_skew"]=train[train.columns[175:200].to_list()].skew(axis=1)
train["sdoif_skew"]=train[train.columns[200:225].to_list()].skew(axis=1)
#########################################################################
train["elevation_q7.5"]=train[train.columns[:25].to_list()].quantile(q=0.75,axis=1)
train["slope_q7.5"]=train[train.columns[25:50].to_list()].quantile(q=0.75,axis=1)
train["aspect_q7.5"]=train[train.columns[50:75].to_list()].quantile(q=0.75,axis=1)
train["placurv_q7.5"]=train[train.columns[75:100].to_list()].quantile(q=0.75,axis=1)
train["procurv_q7.5"]=train[train.columns[100:125].to_list()].quantile(q=0.75,axis=1)
train["lsfactor_q7.5"]=train[train.columns[125:150].to_list()].quantile(q=0.75,axis=1)
train["twi_q7.5"]=train[train.columns[150:175].to_list()].quantile(q=0.75,axis=1)
train["geology_q7.5"]=train[train.columns[175:200].to_list()].quantile(q=0.75,axis=1)
train["sdoif_q7.5"]=train[train.columns[200:225].to_list()].quantile(q=0.75,axis=1)
#########################################################################
train["elevation_q5"]=train[train.columns[:25].to_list()].quantile(q=0.5,axis=1)
train["slope_q5"]=train[train.columns[25:50].to_list()].quantile(q=0.5,axis=1)
train["aspect_q5"]=train[train.columns[50:75].to_list()].quantile(q=0.5,axis=1)
train["placurv_q5"]=train[train.columns[75:100].to_list()].quantile(q=0.5,axis=1)
train["procurv_q5"]=train[train.columns[100:125].to_list()].quantile(q=0.5,axis=1)
train["lsfactor_q5"]=train[train.columns[125:150].to_list()].quantile(q=0.5,axis=1)
train["twi_q5"]=train[train.columns[150:175].to_list()].quantile(q=0.5,axis=1)
train["geology_q5"]=train[train.columns[175:200].to_list()].quantile(q=0.5,axis=1)
train["sdoif_q5"]=train[train.columns[200:225].to_list()].quantile(q=0.5,axis=1)
#########################################################################
train["elevation_q2.5"]=train[train.columns[:25].to_list()].quantile(q=0.25,axis=1)
train["slope_q2.5"]=train[train.columns[25:50].to_list()].quantile(q=0.25,axis=1)
train["aspect_q2.5"]=train[train.columns[50:75].to_list()].quantile(q=0.25,axis=1)
train["placurv_q2.5"]=train[train.columns[75:100].to_list()].quantile(q=0.25,axis=1)
train["procurv_q2.5"]=train[train.columns[100:125].to_list()].quantile(q=0.25,axis=1)
train["lsfactor_q2.5"]=train[train.columns[125:150].to_list()].quantile(q=0.25,axis=1)
train["twi_q2.5"]=train[train.columns[150:175].to_list()].quantile(q=0.25,axis=1)
train["geology_q2.5"]=train[train.columns[175:200].to_list()].quantile(q=0.25,axis=1)
train["sdoif_q2.5"]=train[train.columns[200:225].to_list()].quantile(q=0.25,axis=1)
#########################################################################
train.drop(col, axis=1, inplace=True)

In [141]:

test["elevation_mean"]=test[test.columns[:25].to_list()].mean(axis=1)
test["slope_mean"]=test[test.columns[25:50].to_list()].mean(axis=1)
test["aspect_mean"]=test[test.columns[50:75].to_list()].mean(axis=1)
test["placurv_mean"]=test[test.columns[75:100].to_list()].mean(axis=1)
test["procurv_mean"]=test[test.columns[100:125].to_list()].mean(axis=1)
test["lsfactor_mean"]=test[test.columns[125:150].to_list()].mean(axis=1)
test["twi_mean"]=test[test.columns[150:175].to_list()].mean(axis=1)
test["geology_mean"]=test[test.columns[175:200].to_list()].mean(axis=1)
test["sdoif_mean"]=test[test.columns[200:225].to_list()].mean(axis=1)
########################################################################
test["elevation_count"]=test[test.columns[:25].to_list()].count(axis=1)
test["slope_count"]=test[test.columns[25:50].to_list()].count(axis=1)
test["aspect_count"]=test[test.columns[50:75].to_list()].count(axis=1)
test["placurv_count"]=test[test.columns[75:100].to_list()].count(axis=1)
test["procurv_count"]=test[test.columns[100:125].to_list()].count(axis=1)
test["lsfactor_count"]=test[test.columns[125:150].to_list()].count(axis=1)
test["twi_count"]=test[test.columns[150:175].to_list()].count(axis=1)
test["geology_count"]=test[test.columns[175:200].to_list()].count(axis=1)
test["sdoif_count"]=test[test.columns[200:225].to_list()].count(axis=1)
#########################################################################
test["elevation_std"]=test[test.columns[:25].to_list()].std(axis=1)
test["slope_std"]=test[test.columns[25:50].to_list()].std(axis=1)
test["aspect_std"]=test[test.columns[50:75].to_list()].std(axis=1)
test["placurv_std"]=test[test.columns[75:100].to_list()].std(axis=1)
test["procurv_std"]=test[test.columns[100:125].to_list()].std(axis=1)
test["lsfactor_std"]=test[test.columns[125:150].to_list()].std(axis=1)
test["twi_std"]=test[test.columns[150:175].to_list()].std(axis=1)
test["geology_std"]=test[test.columns[175:200].to_list()].std(axis=1)
test["sdoif_std"]=test[test.columns[200:225].to_list()].std(axis=1)
#########################################################################
test["elevation_min"]=test[test.columns[:25].to_list()].min(axis=1)
test["slope_min"]=test[test.columns[25:50].to_list()].min(axis=1)
test["aspect_min"]=test[test.columns[50:75].to_list()].min(axis=1)
test["placurv_min"]=test[test.columns[75:100].to_list()].min(axis=1)
test["procurv_min"]=test[test.columns[100:125].to_list()].min(axis=1)
test["lsfactor_min"]=test[test.columns[125:150].to_list()].min(axis=1)
test["twi_min"]=test[test.columns[150:175].to_list()].min(axis=1)
test["geology_min"]=test[test.columns[175:200].to_list()].min(axis=1)
test["sdoif_min"]=test[test.columns[200:225].to_list()].min(axis=1)
#########################################################################
test["elevation_max"]=test[test.columns[:25].to_list()].max(axis=1)
test["slope_max"]=test[test.columns[25:50].to_list()].max(axis=1)
test["aspect_max"]=test[test.columns[50:75].to_list()].max(axis=1)
test["placurv_max"]=test[test.columns[75:100].to_list()].max(axis=1)
test["procurv_max"]=test[test.columns[100:125].to_list()].max(axis=1)
test["lsfactor_max"]=test[test.columns[125:150].to_list()].max(axis=1)
test["twi_max"]=test[test.columns[150:175].to_list()].max(axis=1)
test["geology_max"]=test[test.columns[175:200].to_list()].max(axis=1)
test["sdoif_max"]=test[test.columns[200:225].to_list()].max(axis=1)
#########################################################################
test["elevation_kurt"]=test[test.columns[:25].to_list()].kurt(axis=1)
test["slope_kurt"]=test[test.columns[25:50].to_list()].kurt(axis=1)
test["aspect_kurt"]=test[test.columns[50:75].to_list()].kurt(axis=1)
test["placurv_kurt"]=test[test.columns[75:100].to_list()].kurt(axis=1)
test["procurv_kurt"]=test[test.columns[100:125].to_list()].kurt(axis=1)
test["lsfactor_kurt"]=test[test.columns[125:150].to_list()].kurt(axis=1)
test["twi_kurt"]=test[test.columns[150:175].to_list()].kurt(axis=1)
test["geology_kurt"]=test[test.columns[175:200].to_list()].kurt(axis=1)
test["sdoif_kurt"]=test[test.columns[200:225].to_list()].kurt(axis=1)
#########################################################################
test["elevation_skew"]=test[test.columns[:25].to_list()].skew(axis=1)
test["slope_skew"]=test[test.columns[25:50].to_list()].skew(axis=1)
test["aspect_skew"]=test[test.columns[50:75].to_list()].skew(axis=1)
test["placurv_skew"]=test[test.columns[75:100].to_list()].skew(axis=1)
test["procurv_skew"]=test[test.columns[100:125].to_list()].skew(axis=1)
test["lsfactor_skew"]=test[test.columns[125:150].to_list()].skew(axis=1)
test["twi_skew"]=test[test.columns[150:175].to_list()].skew(axis=1)
test["geology_skew"]=test[test.columns[175:200].to_list()].skew(axis=1)
test["sdoif_skew"]=test[test.columns[200:225].to_list()].skew(axis=1)
#########################################################################
test["elevation_q7.5"]=test[test.columns[:25].to_list()].quantile(q=0.75,axis=1)
test["slope_q7.5"]=test[test.columns[25:50].to_list()].quantile(q=0.75,axis=1)
test["aspect_q7.5"]=test[test.columns[50:75].to_list()].quantile(q=0.75,axis=1)
test["placurv_q7.5"]=test[test.columns[75:100].to_list()].quantile(q=0.75,axis=1)
test["procurv_q7.5"]=test[test.columns[100:125].to_list()].quantile(q=0.75,axis=1)
test["lsfactor_q7.5"]=test[test.columns[125:150].to_list()].quantile(q=0.75,axis=1)
test["twi_q7.5"]=test[test.columns[150:175].to_list()].quantile(q=0.75,axis=1)
test["geology_q7.5"]=test[test.columns[175:200].to_list()].quantile(q=0.75,axis=1)
test["sdoif_q7.5"]=test[test.columns[200:225].to_list()].quantile(q=0.75,axis=1)
#########################################################################
test["elevation_q5"]=test[test.columns[:25].to_list()].quantile(q=0.5,axis=1)
test["slope_q5"]=test[test.columns[25:50].to_list()].quantile(q=0.5,axis=1)
test["aspect_q5"]=test[test.columns[50:75].to_list()].quantile(q=0.5,axis=1)
test["placurv_q5"]=test[test.columns[75:100].to_list()].quantile(q=0.5,axis=1)
test["procurv_q5"]=test[test.columns[100:125].to_list()].quantile(q=0.5,axis=1)
test["lsfactor_q5"]=test[test.columns[125:150].to_list()].quantile(q=0.5,axis=1)
test["twi_q5"]=test[test.columns[150:175].to_list()].quantile(q=0.5,axis=1)
test["geology_q5"]=test[test.columns[175:200].to_list()].quantile(q=0.5,axis=1)
test["sdoif_q5"]=test[test.columns[200:225].to_list()].quantile(q=0.5,axis=1)
#########################################################################
test["elevation_q2.5"]=test[test.columns[:25].to_list()].quantile(q=0.25,axis=1)
test["slope_q2.5"]=test[test.columns[25:50].to_list()].quantile(q=0.25,axis=1)
test["aspect_q2.5"]=test[test.columns[50:75].to_list()].quantile(q=0.25,axis=1)
test["placurv_q2.5"]=test[test.columns[75:100].to_list()].quantile(q=0.25,axis=1)
test["procurv_q2.5"]=test[test.columns[100:125].to_list()].quantile(q=0.25,axis=1)
test["lsfactor_q2.5"]=test[test.columns[125:150].to_list()].quantile(q=0.25,axis=1)
test["twi_q2.5"]=test[test.columns[150:175].to_list()].quantile(q=0.25,axis=1)
test["geology_q2.5"]=test[test.columns[175:200].to_list()].quantile(q=0.25,axis=1)
test["sdoif_q2.5"]=test[test.columns[200:225].to_list()].quantile(q=0.25,axis=1)
#########################################################################
test.drop(col, axis=1, inplace=True)

In [142]:
y_train = train[['Label']]
df_train=train.drop(['Label'], axis=1)
cols=df_train.columns

In [143]:
train[train.columns[1:]]

,elevation_mean,slope_mean,aspect_mean,placurv_mean,procurv_mean,lsfactor_mean,twi_mean,geology_mean,sdoif_mean,elevation_count,...,sdoif_q5,elevation_q2.5,slope_q2.5,aspect_q2.5,placurv_q2.5,procurv_q2.5,lsfactor_q2.5,twi_q2.5,geology_q2.5,sdoif_q2.5
0,119.44,36.830369,116.864561,0.012659,-0.008915,9.013694,3.466513,3.00,1.281733,25,...,1.281730,115.0,33.85452,113.1986,0.005337,-0.023553,8.045186,3.154479,3.0,1.281698
1,156.20,28.524667,184.633436,0.010663,-0.003822,8.013825,4.660537,3.00,1.359578,25,...,1.359579,153.0,26.56505,180.0000,0.005583,-0.008197,7.111054,4.441799,3.0,1.359547
2,162.56,38.336258,284.942436,0.028086,0.022452,10.958018,4.227799,2.00,1.365054,25,...,1.365055,158.0,31.31116,270.0000,0.013423,0.008324,9.387614,3.363745,2.0,1.365018
3,76.40,19.271319,173.076448,0.029064,0.008895,3.785720,3.737841,2.00,1.100818,25,...,1.100826,75.0,16.69924,161.5650,0.022927,0.005753,3.194233,3.467204,2.0,1.100731
4,109.16,28.257938,125.512572,-0.004572,-0.007679,7.742521,4.542060,3.80,1.284221,25,...,1.284222,106.0,26.56505,120.9638,-0.009951,-0.010910,7.337839,4.083162,2.0,1.284092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,198.04,20.321682,118.451976,0.011845,-0.001129,4.039822,4.002014,2.36,1.272181,25,...,1.272174,196.0,15.79317,111.8014,0.002935,-0.016208,2.524856,3.677230,2.0,1.272011
10860,424.32,28.673109,150.606208,0.019535,0.012576,6.449958,3.481891,3.00,1.331094,25,...,1.331096,421.0,26.56505,135.0000,0.009280,0.007461,5.375127,3.211934,3.0,1.331052
10861,335.44,40.161622,136.401204,0.003379,0.004456,10.775079,3.728304,3.00,1.298318,25,...,1.298319,332.0,37.99073,130.6013,-0.010156,-0.003635,10.294480,3.254569,3.0,1.298297
10862,96.64,35.370753,253.184260,-0.001736,0.012439,9.183515,3.802240,2.00,1.355144,25,...,1.355141,93.0,33.85452,243.4350,-0.014070,0.003979,8.591082,3.384373,2.0,1.355061


In [144]:
from imblearn.over_sampling import SMOTE
sampler = SMOTE(k_neighbors=5, random_state=11)
df ,y = sampler.fit_resample(train[train.columns[1:]] ,train["Label"])
df["y"]=y

In [145]:
# from imblearn.over_sampling import RandomOverSampler
# sampler = RandomOverSampler(random_state=11)
# df ,y = sampler.fit_resample(train[train.columns[1:]] ,train["Label"])

**without scaling 0.74 with scaling 0.73**

In [146]:
# from sklearn.preprocessing import RobustScaler
# ss = RobustScaler()
# df_train = pd.DataFrame(ss.fit_transform(df_train), columns=df_train.columns)
# test = pd.DataFrame(ss.transform(test), columns=test.columns)


In [150]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold,StratifiedKFold ,GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn import  ensemble
def get_model(Name='lgbm') :
    if Name=='lgbm' :
      return LGBMClassifier(**{ 'boosting_type': 'gbdt','objective': 'binary','metric': 'auc',
                                'n_estimators': 500,'colsample_bytree' : 0.8,
                                'seed': 42,'silent':False,
                                'early_stopping_rounds': 100,
                                'learning_rate' : 0.05,'num_iterations': 1500,
                                'max_depth' :8 ,'num_leaves' : 150,
                                'max_bins': 85,'min_data_in_leaf':30 })
test_ = test[cols]
Model_Name = "lgbm"
folds = StratifiedKFold(n_splits=15, shuffle=True, random_state=5168)
oofs  = np.zeros((len(df[cols])))
test_predictions = np.zeros((len(test)))
tst_predd = np.zeros((len(test)))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df, df['y'])):
    X_train, y_train = df.iloc[trn_idx][cols], df.iloc[trn_idx]['y']
    X_valid, y_valid = df.iloc[val_idx][cols], df.iloc[val_idx]['y']
#     X_trn, y_trn = X.iloc[trn_idx], y.iloc[trn_idx]
#     X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    
    clf = get_model(Name=Model_Name)
    clf.fit(X_train, y_train, eval_set = [(X_valid, y_valid)],
            verbose =100, early_stopping_rounds = 500)
    
    vp = clf.predict(X_valid)
    oofs[val_idx] = vp
    val_score = mean_squared_error((vp), (y_valid),squared=True)
    print(4*'-- -- -- --')
    print(f'Fold {fold_+1} Val score: {val_score}')
    print(4*'-- -- -- --')
    
    tp = clf.predict(test_)
    test_predictions += tp / folds.n_splits
    tst_predd += tp

  
print()
print(3*'###',10*"^",3*'###')
print(mean_squared_error(df["y"], oofs,squared=True))
print("Model training")
# clf.fit(X_train, y_train )

[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] early_stopping_round is set=500, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=500
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 7605, number of negative: 7604
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020563 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6699
[LightGBM] [Info] Number of data points in the train set: 15209, number of used features: 81
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] early_stopping_round is set=500, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=500

In [152]:
from sklearn.metrics import classification_report
print(classification_report(df["y"], oofs))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      8148
           1       0.91      0.93      0.92      8148

    accuracy                           0.92     16296
   macro avg       0.92      0.92      0.92     16296
weighted avg       0.92      0.92      0.92     16296



              precision    recall  f1-score   support

           0       0.93      0.91      0.92      8148
           1       0.91      0.93      0.92      8148

    accuracy                           0.92     16296
   macro avg       0.92      0.92      0.92     16296
weighted avg       0.92      0.92      0.92     16296


           precision    recall  f1-score   support

           0       0.89      0.94      0.91      8148
           1       0.77      0.66      0.71      2716

    accuracy                           0.87     10864
   macro avg       0.83      0.80      0.81     10864
weighted avg       0.86      0.87      0.86     10864

In [ ]:
np.unique(test_predictions)

In [ ]:
# len(test_predictions)

In [ ]:
# cols=test.columns
# predict = clf.predict_proba(test[cols])

In [ ]:
# predict

In [ ]:
# l=[]
# for i in predict:
#     l.append(np.argmax(i))

In [ ]:
for i in range(len(test_predictions)):
    if test_predictions[i]>=0.5:
        test_predictions[i]=1
    else:
        test_predictions[i]=0

In [ ]:
sub=pd.DataFrame({"Sample_ID":id_,"Label":test_predictions})

In [ ]:
sub

In [ ]:
from IPython.display import FileLink
def create_submission(submission_file, submission_name):
    submission_file.to_csv(submission_name+".csv",index=False)
    return FileLink(submission_name+".csv")
create_submission(sub, "sol")

#### 